In [1]:
import numpy as np
import pykasso.tools as pkt

### GEOLOGY
gslib_geology = pkt.GSLIB('test/inputs/betteraz_2D_z430.gslib')
geology = gslib_geology.export_column('intersect3', (300,300))
geology = np.repeat(geology[:, :, np.newaxis], 20, axis=2)

# for i in range(0,290,10):
#     geology[:, i+0, 19] = 0
#     geology[:, i+1, 19] = 0
#     geology[:, i+2, 19] = 0

### TOPOGRAPHY
topography = np.full((300,300), 350)
topography[:, 100:200] = 250
topography[100:200, :] = 250

### BEDROCK ELEVATION
bedrock_elevation = np.full((300,300), 50)
bedrock_elevation[:, 10:100] = 25

### WATER LEVEL ELEVATION
water_level_elevation = np.full((300,300), 200)
water_level_elevation[200:299, :] = 150

sks_settings = {
    'grid' : {
        'x0' : 572510,
        'y0' : 250010,
        'z0' : 100,
        'nx' : 300,
        'ny' : 300,
        'nz' : 20,
        'dx' : 20,
        'dy' : 20,
        'dz' : 20
    },
    'domain' : {
        'delimitation' : 'test/inputs/betteraz_polygon.txt',
        # 'topography'   : topography,
        # 'bedrock'      : bedrock_elevation,
        'water_level'  : water_level_elevation,
    },
    # 'geology' : {
    #     # 'data' : 'test/inputs/betteraz_2D_z430.gslib',
    #     'data' : geology,
    #     'costs': {1: 0.5, 2: 0.7}
    # },
    # 'faults' : {
    #     'data' : 'test/inputs/faults.png',
    #     'axis': 'z',
        # 'cost': {1: 0.3, 2: 0.4}
    # },
    'fmm' : {
        'algorithm' : 'Riemann3', #'Isotropic3',
        'costs' : {
            'ratio' : 0.5
        },
    },
    'sks' : {
        'seed' : 0,
        'domain_from_geology' : False,
    },
    'verbosity' : {
        'logging'  : 0,
        'agd'      : 0,
        'karstnet' : 0
    },
    'outlets' : {
        'number'     : 3,
        # 'data'       : 'test/inputs/points.txt',
        'data'       : '', #[[100,100], [200,200], [300,300], [4000,4000]],
        # 'data' : '',
        # 'shuffle'    : False,
        'importance' : [1,1,1],
        'mode' : 'surface_down',
        # 'geology'    : [1],
        'seed' : 2,
        # 'x' : 'lambda : 1500 + 1500 * rng.random() * grid.dx',
        # 'y' : 'lambda : 1500 + 1500 * rng.random() * grid.dy',
        # 'z' : 'lambda : grid.z0'
    },
    'inlets' : {
        'number' : 30,
        # 'data'       : [],
        # 'shuffle'    : False,
        'per_outlet' : [5,5,5],
        'importance' : [1,1,1],
        'mode' : 'surface_up',
        # 'geology'    : [2],
        'seed' : 1,
        # 'z' : 'lambda x, y : x + y'
    },
    # 'fractures': {
    #     'data': 'generator',
    #     'axis': 'z',
    #     'seed': 534106,
    #     'settings': {
    #         'family_01': {
    #             'alpha'        : 2,
    #             'density'      : 0.00001,
    #             'orientation'  : [340, 20],
    #             'dip'          : [80, 90],
    #             'length'       : [1000, 2000],
    #         }, 
    #         'family_02': {
    #             'alpha'        : 2,
    #             'density'      : 0.00001,
    #             'orientation'  : [340, 20],
    #             'dip'          : [0, 10],
    #             'length'       : [1000, 2000],
    #         }
        # }
    # },
}

In [2]:
import numpy as np
import pykasso as pk
pk.create_project('test')

debug_level = {
    'model'      : 4,
    # 'simulation' : 0,
    # 'iteration'  : 2,
}

for _ in range(1):
    sim = pk.SKS(debug_level=debug_level, sks_settings=sks_settings)
    # sim = pk.SKS(sks_settings=sks_settings)
    # sim = pk.SKS(debug_level=debug_level)
    # sim = pk.SKS()
    sim.build_model()
    sim.compute_karst_network()

CAUTION: You are using the development version of this package.

 This network contains  1  connected components

Computing:
 - mean length,cv length,length entropy,mean tortuosity
 - orientation entropy,aspl,cpd,md,cv degree,cvd


In [ ]:
import pykasso.visualization as pkv
# pkv.show_array(sim.domain.phreatic['pheatic_zone']) 
pkv.show_array(sim.maps['nodes']) 
# TODO affichage des coleurs par numero 
# pkv.show_array(sim.conceptual_model.data_volume)

In [ ]:
# import matplotlib.pyplot as plt
# f = plt.figure()
# plt.imshow(sim.domain.data_surface)
# plt.colorbar()

In [5]:
import numpy as np
import pykasso.visualization as pkv

settings = {
    # 'show_grid' : True,
    'ghost'     : [0],
    # 'slice'     : True,
    'domain'    : True,
    'inlets'    : True,
    'outlets'   : True,
    # 'iteration' : 0
    # 'tracers'   : True,
}
pkv.show(sim, feature='karst', engine='pyvista', settings=settings)
# bedrock_elevation
# # TODO - show a specific fracture family
# ajouter 'outlets' et 'inlets'

In [3]:
import numpy as np
import pykasso.visualization as pkv

steps = {
    1 : 'model',
    2 : 'fmm'
}
choice = 2

settings = {
    'iterations' : [0,2,4,6,8]
}

pkv.debug(sim, step=steps[choice], engine='pyvista', settings=settings)

# TODO

In [ ]:
### pyKasso

    ### general
    # -> Tests pour tous les attributs de SKS_SETTINGS : verbosity / piezometric_level
    # -> implémenter les volumes  : anisotropie avec pendage ???
    # -> collection d'exemples géométriques
    # -> méthode pour ajouter une étude de cas

    ### documentation / code
    # expliciter les sorties de fonctions

    ### tools - dev
    # 50% cProfile system
    # wrapper? : time process measurements (SKS, initilization, modelisation, each iteration?)

    ### tools - user
    # Geology editor ?
    # Topography editor ?
    # Water surface level editor ?
    
    ### visualization
    # wrapper gif

### futur
# mutli-processing
# add timing with UTC 

# Questions

In [ ]:
### Modeling
# Definition du domaine
# -> 'Domain' : quelles règles pour 'delimitation' / 'bedrock' / 'topography' ???
# -> topo et bedrock n'ont pas le droit de couper verticalement le modèle... ? 
# Bedding / faults / fractures
# -> un objet par couche, ou un objet qui agglomère tout ?

### Fast-marching
# Combien d'itération au total ? 
# Comment 'per_outlet' doit être contrôlé ?

### Management
# Structure du fichier d'output ?
# Noms des fonctions publiques ?